In [1]:
import zipfile
path_to_zip_file = "/content/실습6 데이터.zip"
tar_dir = "/content"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(tar_dir)

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

# 데이터 불러오기
class0 = pd.read_csv('class0.csv', header=None)
class1 = pd.read_csv('class1.csv', header=None)

# 데이터 전처리
X = pd.concat([pd.DataFrame(class0.iloc[:, 1:]), pd.DataFrame(class1.iloc[:, 1:])], axis=0)
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(pd.concat([pd.Series([0] * class0.shape[0]), pd.Series([1] * class1.shape[0])], ignore_index=True).values, dtype=torch.long)

# 데이터 분할
dataset = TensorDataset(X, y)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

# 데이터 로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [24]:

# LSTM 모델 정의
input_size = X.shape[1]
hidden_size = 64
num_layers = 2
num_classes = 2

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1])
        return out


# 모델 인스턴스 생성 및 학습
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 테스트 데이터 평가
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch: {epoch+1}, Accuracy: {accuracy:.2f}%')

RuntimeError: Expected hidden[0] size (2, 1, 64), got [32, 1, 64]